In [21]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms


import torchvision


In [2]:
import visdom
vis  = visdom.Visdom()

Setting up a new session...


In [18]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device =='cuda':
    torch.cuda.manual_seed_all(777)

In [29]:
#parameters
learning_rate = 0.001
training_epochs =15
batch_size = 100

def loss_tracker(loss_plot, loss_value, num):
    # num과 loss_value는 Tensor
    vis.line(X=num, Y=loss_value, win = loss_plot, update='append')

In [24]:

mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                         transform = transforms.ToTensor(),
                         download =True)
mnist_test = dsets.MNIST(root = "MNIST_data/",
                        train= False,
                        transform =transforms.ToTensor(),
                        download =True)

data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                          batch_size = batch_size,
                                          shuffle = True,
                                            drop_last=True
                                          )


In [25]:


class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.layer1= nn.Sequential(
        nn.Conv2d(1,32,kernel_size= 3 , stride =1, padding =1),
        nn.ReLU(),
        nn.MaxPool2d(2)
        )
        
        self.layer2 = nn.Sequential(
        nn.Conv2d(32,64, kernel_size= 3, stride =1, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(2)
        )
        
        self.fc= nn.Linear(7*7*64, 10, bias = True)
        torch.nn.init.xavier_uniform_(self.fc.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        
        out = out.view(out.size(0),-1)
        out = self.fc(out)
        return out


In [27]:

model = CNN().to(device)

# 임의의 값을 집어넣어서 모델 확인
value = (torch.Tensor(1,1,28,28)).to(device)
print( (model(value)).shape )


torch.Size([1, 10])


In [30]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

# cost function plot
loss_plt = vis.line(Y=torch.Tensor(1).zero_(),opts=dict(title='loss_tracker', legend=['loss'], showlegend=True))

#training
total_batch = len(data_loader)

for epoch in range(training_epochs):
    avg_cost = 0
    for X, Y in data_loader:
        X = X.to(device) #이미지
        Y = Y.to(device ) # label
        
        optimizer.zero_grad()
        hypothesis = model(X)
        
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost /total_batch
    print('[Epoch : {}] cost = {}'.format(epoch+1, avg_cost))
    loss_tracker(loss_plt, torch.Tensor([avg_cost]), torch.Tensor([epoch]))


[Epoch : 1] cost = 0.07081247866153717
[Epoch : 2] cost = 0.04556210711598396


KeyboardInterrupt: 

In [ ]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test),1,28,28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction,1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuray: ',accuracy.Item() )

In [31]:
vis.close(env = 'main')

''

[WinError 10054] An existing connection was forcibly closed by the remote host
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[